## Observed data, clustering with KMeans

In [1]:
import numpy as np
import pandas as pd
import os
import cluster_data
import cluster_plotter
from cluster_data import run_clustering, normalize_data, unnormalize
from cluster_plotter import ClusterPlotter
from clustering_utils import ClusterData
from kmeans import k_means
import getdata

# Define observed files
uncorr_obs_files = {
    year: f"ogs{year}01_12_det.ele_ucorr" if year != 2002 else f"ogs{year}08_12_det.ele_ucorr"
    for year in range(2002, 2024) if year != 2018  # Exclude 2018 if missing
}

print(uncorr_obs_files)

standard_year_ranges = {f"{start}-{start + 3}": np.arange(start, start + 4) for start in [2002, 2006, 2010, 2014]}
standard_year_ranges["2019-2023"] = np.arange(2019, 2024)

running_ranges = cluster_data.generate_running_year_ranges(2002, 2023, 4)
year_ranges = running_ranges

# Bin the observed data
binned_data = cluster_data.bin_observed_data(uncorr_obs_files, year_ranges, print_res=False)

# Output directory for plots
plot_dir = "kmeans_observed_running_bins"
os.makedirs(plot_dir, exist_ok=True)
cluster_plotter.clear_directory(plot_dir)

for cluster_data, year_range in binned_data:
    print(f"\nRunning K-Means for Year Range: {year_range}")
    
    data_array = np.array([cluster_data.inc, cluster_data.raan]).T
    normalized_data, data_min, data_max = normalize_data(data_array)
    
    results = []
    for k in [3, 4, 5, 6, 7]:
        result_kmeans, time_kmeans, n_clusters_kmeans, points_per_cluster_kmeans, metrics_kmeans = run_clustering(
            k_means, "K-Means", normalized_data, data_min, data_max, k, plot=False
        )

        unnormalized_data, cluster_centers = unnormalize(result_kmeans.data, result_kmeans.cluster_centers, data_min, data_max)
        plotter = ClusterPlotter(unnormalized_data, result_kmeans.labels, cluster_centers)
        plotter.clusters_2d_plot(f"K-Means: years = {year_range}, k = {k}", os.path.join(plot_dir, f"kmeans_{year_range}_k{k}.png"))

        results.append({
            "Year Range": year_range, "k": k, "Runtime (s)": f"{time_kmeans:.3f}", "Clusters": n_clusters_kmeans,
            "Points per Cluster": points_per_cluster_kmeans,
            "Davies-Bouldin": f"{metrics_kmeans[0]:.3f}" if metrics_kmeans else None,
            "Calinski-Harabasz": f"{metrics_kmeans[1]:.3f}" if metrics_kmeans else None,
            "Dunn Index": f"{metrics_kmeans[2]:.3f}" if metrics_kmeans else None,
            "Silhouette Score": f"{metrics_kmeans[3]:.3f}" if metrics_kmeans else None
        })
    
    display(pd.DataFrame(results))

KeyboardInterrupt: 

## Observed data, clustering with DBSCAN

In [ ]:
import numpy as np
import pandas as pd
import os
import cluster_data
import cluster_plotter
from cluster_data import run_clustering, normalize_data, unnormalize
from cluster_plotter import ClusterPlotter
from clustering_utils import ClusterData
from DBSCAN import dbscan_clustering  # Assuming you have a DBSCAN implementation
import getdata

# Define observed files
uncorr_obs_files = {
    year: f"ogs{year}01_12_det.ele_ucorr" if year != 2002 else f"ogs{year}08_12_det.ele_ucorr"
    for year in range(2002, 2024) if year != 2018  # Exclude 2018 if missing
}

print(uncorr_obs_files)

standard_year_ranges = {f"{start}-{start + 3}": np.arange(start, start + 4) for start in [2002, 2006, 2010, 2014]}
standard_year_ranges["2019-2023"] = np.arange(2019, 2024)

running_ranges = cluster_data.generate_running_year_ranges(2002, 2023, 4)
year_ranges = standard_year_ranges

# Bin the observed data
binned_data = cluster_data.bin_observed_data(uncorr_obs_files, year_ranges, print_res=False)

# Output directory for plots
plot_dir = "dbscan_observed_normal_bins"
os.makedirs(plot_dir, exist_ok=True)
cluster_plotter.clear_directory(plot_dir)

for cluster_data, year_range in binned_data:
    print(f"\nRunning DBSCAN for Year Range: {year_range}")
    
    data_array = np.array([cluster_data.inc, cluster_data.raan]).T
    normalized_data, data_min, data_max = normalize_data(data_array)
    
    results = []
    eps_values = [0.02, 0.01, 0.015, 0.05]
    min_samples_values = [10, 15, 20, 25]
    for eps in eps_values:
        for min_samples in min_samples_values:
            result_dbscan, time_dbscan, n_clusters_dbscan, points_per_cluster_dbscan, metrics_dbscan = run_clustering(
                dbscan_clustering, "DBSCAN", normalized_data, data_min, data_max, plot=False, eps=eps, min_samples=min_samples
            )

            unnormalized_data, cluster_centers = unnormalize(result_dbscan.data, result_dbscan.cluster_centers, data_min, data_max)
            plotter = ClusterPlotter(unnormalized_data, result_dbscan.labels, cluster_centers)
            plotter.clusters_2d_plot(
                f"DBSCAN: years = {year_range}, eps = {eps}, min_samples = {min_samples}",
                os.path.join(plot_dir, f"dbscan_{year_range}_eps{eps}_min{min_samples}.png")
            )

            results.append({
                "Year Range": year_range, "eps": eps, "min_samples": min_samples, "Runtime (s)": f"{time_dbscan:.3f}",
                "Clusters": n_clusters_dbscan, "Points per Cluster": points_per_cluster_dbscan,
                "Davies-Bouldin": f"{metrics_dbscan[0]:.3f}" if metrics_dbscan else None,
                "Calinski-Harabasz": f"{metrics_dbscan[1]:.3f}" if metrics_dbscan else None,
                "Dunn Index": f"{metrics_dbscan[2]:.3f}" if metrics_dbscan else None,
                "Silhouette Score": f"{metrics_dbscan[3]:.3f}" if metrics_dbscan else None
            })
    
    display(pd.DataFrame(results))


{2002: 'ogs200208_12_det.ele_ucorr', 2003: 'ogs200301_12_det.ele_ucorr', 2004: 'ogs200401_12_det.ele_ucorr', 2005: 'ogs200501_12_det.ele_ucorr', 2006: 'ogs200601_12_det.ele_ucorr', 2007: 'ogs200701_12_det.ele_ucorr', 2008: 'ogs200801_12_det.ele_ucorr', 2009: 'ogs200901_12_det.ele_ucorr', 2010: 'ogs201001_12_det.ele_ucorr', 2011: 'ogs201101_12_det.ele_ucorr', 2012: 'ogs201201_12_det.ele_ucorr', 2013: 'ogs201301_12_det.ele_ucorr', 2014: 'ogs201401_12_det.ele_ucorr', 2015: 'ogs201501_12_det.ele_ucorr', 2016: 'ogs201601_12_det.ele_ucorr', 2017: 'ogs201701_12_det.ele_ucorr', 2019: 'ogs201901_12_det.ele_ucorr', 2020: 'ogs202001_12_det.ele_ucorr', 2021: 'ogs202101_12_det.ele_ucorr', 2022: 'ogs202201_12_det.ele_ucorr', 2023: 'ogs202301_12_det.ele_ucorr'}

Running DBSCAN for Year Range: 2002-2005

DBSCAN result:


Running DBSCAN:   0%|          | 0/3 [00:00<?, ?step/s]

Runtime for dbscan_clustering: 0.047153 seconds


Running DBSCAN: 100%|██████████| 3/3 [00:01<00:00,  2.88step/s]


Plot saved as: dbscan_observed_normal_bins\dbscan_2002-2005_eps0.02_min10.png

DBSCAN result:


...

Running DBSCAN:   0%|          | 0/3 [00:00<?, ?step/s]

Runtime for dbscan_clustering: 0.021588 seconds


Running DBSCAN: 100%|██████████| 3/3 [00:00<00:00,  9.59step/s]


Plot saved as: dbscan_observed_normal_bins\dbscan_2019-2023_eps0.015_min15.png


,Year Range,eps,min_samples,Runtime (s),Clusters,Points per Cluster,Davies-Bouldin,Calinski-Harabasz,Dunn Index,Silhouette Score
0,2019-2023,0.020,10,0.015,26,"{0: 38, 1: 15, 2: 1189, 3: 17, 4: 13, 5: 15, 6...",2.139,65.836,0.001,-0.332
1,2019-2023,0.020,15,0.017,16,"{0: 32, 1: 75, 2: 59, 3: 153, 4: 15, 5: 17, 6:...",1.566,94.720,0.001,-0.112
2,2019-2023,0.010,10,0.031,22,"{0: 17, 1: 12, 2: 13, 3: 13, 4: 12, 5: 16, 6: ...",1.496,25.489,0.000,-0.467
3,2019-2023,0.010,15,0.012,10,"{0: 62, 1: 36, 2: 248, 3: 237, 4: 47, 5: 45, 6...",1.690,27.679,0.001,-0.235
4,2019-2023,0.015,10,0.010,33,"{0: 25, 1: 49, 2: 18, 3: 31, 4: 10, 5: 36, 6: ...",1.648,44.574,0.001,-0.279
5,2019-2023,0.015,15,0.022,10,"{0: 29, 1: 26, 2: 22, 3: 93, 4: 19, 5: 141, 6:...",1.624,65.396,0.001,-0.286


In [5]:
import cluster_data
from cluster_data import run_clustering, normalize_data, unnormalize
import numpy as np
import pandas as pd
import os
import cluster_plotter
from DBSCAN import dbscan_clustering
from cluster_plotter import ClusterPlotter
from clustering_utils import ClusterData

uncorr_obs_files = {
    year: f"ogs{year}01_12_det.ele_ucorr" if year != 2002 else f"ogs{year}08_12_det.ele_ucorr"
    for year in range(2002, 2024) if year != 2018  # Exclude 2018 if missing
}

print(uncorr_obs_files)

standard_year_ranges = {f"{start}-{start + 3}": np.arange(start, start + 4) for start in [2002, 2006, 2010, 2014]}
standard_year_ranges["2019-2023"] = np.arange(2019, 2024)

running_ranges = cluster_data.generate_running_year_ranges(2002, 2023, 4)
year_ranges = standard_year_ranges

# Bin the observed data
binned_data = cluster_data.bin_observed_data(uncorr_obs_files, year_ranges, print_res=False)

plot_dir = "dbscan_grid_search_normal_bins_observed"
os.makedirs(plot_dir, exist_ok=True)
cluster_plotter.clear_directory(plot_dir)

results_per_year_range = {}

for cluster_data, year_range in binned_data:  
    print(f"\nRunning DBSCAN for Year Range: {year_range}")

    data_array = np.array([cluster_data.inc, cluster_data.raan]).T
    normalized_data, data_min, data_max = normalize_data(data_array)

    # DBSCAN parameter grid
    eps_values = [0.01, 0.012, 0.015, 0.018, 0.02, 0.023, 0.05]
    min_samples_values = [7, 10, 15, 20, 30]
    
    best_result = None
    best_score = float('-inf')
    
    for eps in eps_values:
        for min_samples in min_samples_values:
            result_dbscan, time_dbscan, n_clusters_dbscan, points_per_cluster_dbscan, metrics_dbscan = run_clustering(
                dbscan_clustering, "DBSCAN", normalized_data, data_min, data_max, plot=False, eps=eps, min_samples=min_samples
            )
            
            silhouette_score = metrics_dbscan[3] if metrics_dbscan and isinstance(metrics_dbscan[3], (int, float)) else None
            noise_points = np.sum(result_dbscan.labels == -1)

            if silhouette_score is not None and silhouette_score > best_score:
                best_score = silhouette_score
                best_result = {
                    "eps": eps,
                    "min_samples": min_samples,
                    "runtime": time_dbscan,
                    "clusters": n_clusters_dbscan,
                    "points_per_cluster": points_per_cluster_dbscan,
                    "noise_points": noise_points,
                    "metrics": metrics_dbscan,
                    "result_dbscan": result_dbscan
                }

    # Plot and store only the best result
    if best_result:
        unnormalized_data, _ = unnormalize(best_result["result_dbscan"].data, None, data_min, data_max)
        plotter = ClusterPlotter(unnormalized_data, best_result["result_dbscan"].labels, None)
        plot_filename = os.path.join(plot_dir, f"dbscan_{year_range}_best.png")
        title = f"DBSCAN Best: {year_range}, eps = {best_result['eps']}, min_samples = {best_result['min_samples']}"
        plotter.clusters_2d_plot(title, plot_filename, show_centers=False)
        
        results_per_year_range[year_range] = best_result
        
        print(f"Best for {year_range}: eps = {best_result['eps']}, min_samples = {best_result['min_samples']}, silhouette = {best_score:.3f}")

final_results = [
    {"Year Range": yr, "Eps": res["eps"], "Min Samples": res["min_samples"], "Silhouette Score": f"{res['metrics'][3]:.3f}", "Noise Points": res["noise_points"]}
    for yr, res in results_per_year_range.items()
]
df = pd.DataFrame(final_results)
display(df)

{2002: 'ogs200208_12_det.ele_ucorr', 2003: 'ogs200301_12_det.ele_ucorr', 2004: 'ogs200401_12_det.ele_ucorr', 2005: 'ogs200501_12_det.ele_ucorr', 2006: 'ogs200601_12_det.ele_ucorr', 2007: 'ogs200701_12_det.ele_ucorr', 2008: 'ogs200801_12_det.ele_ucorr', 2009: 'ogs200901_12_det.ele_ucorr', 2010: 'ogs201001_12_det.ele_ucorr', 2011: 'ogs201101_12_det.ele_ucorr', 2012: 'ogs201201_12_det.ele_ucorr', 2013: 'ogs201301_12_det.ele_ucorr', 2014: 'ogs201401_12_det.ele_ucorr', 2015: 'ogs201501_12_det.ele_ucorr', 2016: 'ogs201601_12_det.ele_ucorr', 2017: 'ogs201701_12_det.ele_ucorr', 2019: 'ogs201901_12_det.ele_ucorr', 2020: 'ogs202001_12_det.ele_ucorr', 2021: 'ogs202101_12_det.ele_ucorr', 2022: 'ogs202201_12_det.ele_ucorr', 2023: 'ogs202301_12_det.ele_ucorr'}

Running DBSCAN for Year Range: 2002-2005

DBSCAN result:


Running DBSCAN:   0%|          | 0/3 [00:00<?, ?step/s]

Runtime for dbscan_clustering: 0.051245 seconds


Running DBSCAN: 100%|██████████| 3/3 [00:01<00:00,  2.91step/s]



DBSCAN result:


...

Running DBSCAN:   0%|          | 0/3 [00:00<?, ?step/s]

Runtime for dbscan_clustering: 0.058764 seconds


Running DBSCAN: 100%|██████████| 3/3 [00:00<00:00,  4.49step/s]
c:\Users\fionu\OneDrive\Dokumente\Daten Fiona\AIUB\Code\clustering\cluster_plotter.py:92: UserWarning: No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.
  plt.legend()


Plot saved as: dbscan_grid_search_normal_bins_observed\dbscan_2019-2023_best.png
Best for 2019-2023: eps = 0.05, min_samples = 15, silhouette = 0.039


,Year Range,Eps,Min Samples,Silhouette Score,Noise Points
0,2002-2005,0.05,30,0.208,521
1,2006-2009,0.05,7,0.239,69
2,2010-2013,0.05,20,0.158,236
3,2014-2017,0.05,20,0.195,255
4,2019-2023,0.05,15,0.039,160
